# **FAISS**

#  ***Loading Data***

In [ ]:
!git clone https://github.com/YoustinaEhab/Arabic-Dataset.git

fatal: destination path 'Arabic-Dataset' already exists and is not an empty directory.


In [ ]:
import json

In [ ]:
with open('Arabic-Dataset/all_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
paragraphs = [entry['paragraph'] for entry in data if 'paragraph' in entry]
metadata = [{'topic': entry['topic']} for entry in data if 'topic' in entry]

In [ ]:
print(f"Loaded {len(paragraphs)} paragraphs!")
print(paragraphs[:2])

Loaded 951 paragraphs!
['دخلت الدولة العثمانية الحرب العالمية الأولى إلى جانب (ألمانيا والنمسا) على أمل استرداد ما اقتنصته منها دول الوفاق (إنجلترا وفرنسا) وإنقاذاً لنفسها ولما تبقى تحت يدها من ولايات عربية من مخططات اقتسام دول الوفاق (إنجلترا/فرنسا) للدولة العثمانية فيما بينهم. وبينما كانت الحرب لا تزال فى أيامها الأولى قام الأسطول العثمانى – مدعماً بقطعتين حربيتين ألمانيتى الأصل- بقصف ميناء أوديسا وميناء سباستبول الروسيين معلنة (الجهاد) المقدس فى نوفمبر ١٩١٤ م، وقد كان لهذا النداء (الجهاد) صدى قويا بين المسلمين، وكان مزعجاً لإنجلترا التى كانت تحكم الهند وجنوب شرق آسيا (حوالى ٥٠ مليوناً من المسلمين على الأقل)، وتسعى لإبقاء تركيا على الحياد ولكن بخروج تركيا عن حيادها وضربها الموانىء الروسية انضم البريطانيون تمامًا إلى الجانب العربى لتحرير العرب من قيودهم وكانت تلك خدعة بريطانية. أما فى البلاد العربية فقد آثار إعلان الجهاد حماساً، ثم لم يلبث أن ساد الحذر بين زعماء العرب، فقد كانت الأزمة بين العرب والترك لا تزال قائمة، بسبب سياسة التتريك التى اتبعتها الدولة العثمانية،والتى تهدف إلى تهميش

In [ ]:
# Create paragraph IDs
paragraphs_id = [str(i) for i in range(len(paragraphs))]

# ***Text to Vector***

In [ ]:
!pip install numpy<2.0 faiss-gpu-cu11 --force-reinstall

/bin/bash: line 1: 2.0: No such file or directory


In [ ]:
#!pip install faiss-cpu==1.11.0

In [ ]:
# !pip uninstall numpy -y
# !pip install numpy --upgrade --force-reinstall

In [ ]:
# !pip install --upgrade --force-reinstall transformers faiss-cpu

In [ ]:
import json
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import numpy as np
import faiss
from copy import deepcopy
import pickle
import os

In [ ]:
def average_pool(last_hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask.unsqueeze(-1).bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
model_id = 'intfloat/multilingual-e5-large-instruct'
#model_id = 'intfloat/multilingual-e5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id).to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:  48%|####7     | 1.02G/2.14G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

In [ ]:
import time
start_time = time.time()

In [ ]:
batch_size = 16
encoded_paragraphs = []

for i in range(0, len(paragraphs), batch_size):
    batch = paragraphs[i:i+batch_size]
    batch_encoding = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**batch_encoding)
        embeddings = average_pool(outputs.last_hidden_state, batch_encoding['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)
    encoded_paragraphs.append(embeddings.cpu().numpy())

encoded_paragraphs = np.vstack(encoded_paragraphs).astype(np.float32)

In [ ]:
embedding_time = time.time() - start_time
print(f"Embedding Time: {embedding_time:.2f} seconds")

Embedding Time: 50.31 seconds


In [ ]:
faiss.normalize_L2(encoded_paragraphs)

dim = encoded_paragraphs.shape[1]

start_time = time.time()

# Create FAISS index (Inner Product = cosine similarity for normalized vectors)
faiss_index = faiss.IndexIDMap(faiss.IndexFlatIP(dim))
faiss_index.add_with_ids(encoded_paragraphs, paragraphs_id)

print(f"FAISS index built with {faiss_index.ntotal} vectors.")
indexing_time = time.time() - start_time
print(f"FAISS Index Creation Time: {indexing_time:.2f} seconds")

FAISS index built with 951 vectors.
FAISS Index Creation Time: 0.00 seconds


In [ ]:
subject_interests = ["الأدب العربي", "التاريخ", "علوم", "الرياضيات"]
question_domain = ['Applying', 'Analyzing', 'Understanding', 'Creating', 'Remembering', 'Evaluating']
difficulty_levels = ["سهل", "متوسط", "صعب"]

# Example interests
selected_interests = ["التاريخ", "متوسط", "Remembering"]

start_time = time.time()

query_string = " ".join(selected_interests)

In [ ]:
query_encoding = tokenizer([query_string], max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
with torch.no_grad():
    query_outputs = model(**query_encoding)
    query_embedding = average_pool(query_outputs.last_hidden_state, query_encoding['attention_mask'])
    query_embedding = F.normalize(query_embedding, p=2, dim=1)
query_embedding_np = query_embedding.cpu().numpy().astype(np.float32)

In [ ]:
query_time = time.time() - start_time
print(f"ChromaDB Query Time: {query_time:.2f} seconds")

ChromaDB Query Time: 0.07 seconds


In [ ]:
# Normalize query embedding (for consistency)
faiss.normalize_L2(query_embedding_np)

# Search top 5 most similar paragraphs
results = faiss_index.search(query_embedding_np, 5)

In [ ]:
import os
import pickle

start_time = time.time()

# ✅ Ensure the directory exists
os.makedirs('/content/faiss-History_docs', exist_ok=True)

# ✅ Save FAISS index and metadata
with open('/content/faiss-History_docs/index.pickle', 'wb') as handle:
    pickle.dump(faiss_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/faiss-History_docs/data.pickle', 'wb') as handle:
    pickle.dump(
        {"data": paragraphs, "ids": paragraphs_id, "metadata": metadata},
        handle,
        protocol=pickle.HIGHEST_PROTOCOL
    )

saving_time = time.time() - start_time
print(f"FAISS Index & Metadata Saving Time: {saving_time:.2f} seconds")


#Load
import pickle

with open('/content/faiss-History_docs/index.pickle','rb') as handle:
  loaded_faiss_index = pickle.load(handle)

with open('/content/faiss-History_docs/data.pickle','rb') as handle:
  loaded_faiss_data = pickle.load(handle)

FAISS Index & Metadata Saving Time: 0.01 seconds


In [ ]:
loaded_faiss_data.keys()

dict_keys(['data', 'ids', 'metadata'])

In [ ]:
total_time = embedding_time + indexing_time + query_time + saving_time
print("\n--- Total Pipeline Timing ---")
print(f"Total Time: {total_time:.2f} seconds")
print(f"  Embedding: {embedding_time:.2f}s")
print(f"  Insertion: {indexing_time:.2f}s")
print(f"  Saving: {saving_time:.2f}s")
print(f"  Query:     {query_time:.2f}s")


--- Total Pipeline Timing ---
Total Time: 50.39 seconds
  Embedding: 50.31s
  Insertion: 0.00s
  Saving: 0.01s
  Query:     0.07s


In [ ]:
distances, indices = results
id_to_data = dict(zip(loaded_faiss_data["ids"], zip(loaded_faiss_data["data"], loaded_faiss_data["metadata"])))

print("Top matching paragraphs and their similarity scores:")
for i, doc_id in enumerate(indices[0]):
    str_doc_id = str(doc_id)  # Ensure ID is a string for lookup
    if str_doc_id in id_to_data:
        paragraph, meta = id_to_data[str_doc_id]
        similarity = distances[0][i]
        print(f"\nParagraph ID: {str_doc_id}")
        print(f"Similarity Score: {similarity:.4f}")
        print(f"Paragraph: {paragraph}")
        print(f"Metadata: {meta}")
    else:
        print(f"Warning: Could not find ID {str_doc_id} in loaded data.")

Top matching paragraphs and their similarity scores:

Paragraph ID: 452
Similarity Score: 0.8626
Paragraph: مراحل الدعوة الإسلامية - المرحلة المكية
Metadata: {'topic': 'الدعوة السرية والجهرية وأساليب المعارضة'}

Paragraph ID: 360
Similarity Score: 0.8502
Paragraph: الموضوع: العصور التاريخية مفاهيم أساسية * عصور تاريخية: هى الفترة الزمنية التى عاش فيها الإنسان بعد معرفة الكتابة. * عصور ما قبل التاريخ: هى الفترة الزمنية التى عاش فيها الإنسان قبل معرفة الكتابة.
Metadata: {'topic': 'مفاهيم العصور التاريخية'}

Paragraph ID: 200
Similarity Score: 0.8284
Paragraph: نظام ملكى: هو نظام حكم وراثى يعتمد على الطبقة الأرستقراطية أى النبلاء.
Metadata: {'topic': 'مفاهيم أساسية - النظام الملكي'}

Paragraph ID: 369
Similarity Score: 0.8271
Paragraph: 2ـ العصور الوسطى وتبدأ منذ سنة ٤٧٦م بسقوط روما وحتى سنة ١٤٥٣م تاريخ سقوط القسطنطينية عاصمة الدولة البيزنطية على يد القائد العثمانى محمد الفاتح، وقد تميز هذا العصر بالعديد من الملامح …
Metadata: {'topic': 'العصور الوسطى'}

Paragraph ID: 375
Similarity Score